## meg preprocess

### overview

#### setup
1. mne install
```
conda install --channel=conda-forge --name=base mamba
mamba create --override-channels --channel=conda-forge --name=mne mne
```

2. freesurfer
- https://surfer.nmr.mgh.harvard.edu/

3. afni
- ttps://afni.nimh.nih.gov/pub/dist/doc/htmldoc/background_install/install_instructs/steps_linux_ubuntu20.html#quick-setup

#### preprocess steps
##### mne
1. cut data
2. denoise
3. BEM model
4. source localization
##### region-based mask
1. downsample with 3dfractionize
2. mask with hcp atlas

#### function
1. data: T1 and raw '.fif'., best with suffix '_raw_tsss_mc.fif'
2. ouput: '.mat' with 180 regions according to hcp glasser atlas.
- https://afni.nimh.nih.gov/pub/dist/atlases/MNI_HCP/MNI_Glasser_HCP_2021_v1.0a/

### setup

#### import necessary packages

In [1]:
# suppress warnings
import warnings
import sys 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import numpy as np #numpy's "nickname" is np
import os
from time import time
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy
import scipy
import csv
import shutil
import joblib
import datetime

import mne
import nibabel as nib
import scipy

# for mask
from nilearn.input_data import NiftiMasker,  MultiNiftiMasker
import nilearn as nil
from scipy import stats

# set log level
# mne.set_log_level("WARNING")
mne.set_log_level("INFO")

# display the plots inline 
%matplotlib inline 
# autosave for every 5 secs
%autosave 5

Autosaving every 5 seconds


#### set environments

In [2]:
project_dir=os.getcwd()
data_dir=os.path.join(project_dir,"data")
freesurfer_dir=os.path.join(project_dir,"freesurfer")
preprocessed_dir=os.path.join(project_dir,"preprocessed")

if not os.path.isdir(preprocessed_dir):
    os.mkdir(preprocessed_dir)

subject_name="sub-xst"
date_string="20231013"
run_name="run01"

preprocessed_subject_path_str = os.path.join(preprocessed_dir, subject_name)


#### visualize raw data
https://www.nmr.mgh.harvard.edu/mne/0.14/auto_tutorials/plot_visualize_raw.html

In [3]:
raw_meg_data_file_name = "run1_tsss_mc.fif"
raw_meg_data_file_path_str = os.path.join(data_dir, subject_name, date_string)
raw = mne.io.read_raw_fif(os.path.join(raw_meg_data_file_path_str, raw_meg_data_file_name), preload=True, allow_maxshield=True)

Opening raw data file /home/brain/Workspace/meg_driving/meg_preprocess/data/sub-xst/20231013/run1_tsss_mc.fif...
    Range : 18000 ... 341999 =     18.000 ...   341.999 secs
Ready.
Reading 0 ... 323999  =      0.000 ...   323.999 secs...


In [ ]:
raw.plot(block=True)

# manually find start and end square wave singal
start_time = 16 # second
end_time = 318

In [4]:
start_time = 16 # second

data_length = 5 * 60 # second

raw_meg_data_file_name = "run1_tsss_mc.fif"
raw_meg_data_file_path_str = os.path.join(data_dir, subject_name, date_string)

raw = mne.io.read_raw_fif(os.path.join(raw_meg_data_file_path_str, raw_meg_data_file_name), preload=True, allow_maxshield=True)
raw = raw.crop(tmin=start_time, tmax=start_time + data_length)

cut_meg_data_file_name = "data_run1_tsss_mc.fif"
raw.save(os.path.join(raw_meg_data_file_path_str, cut_meg_data_file_name), overwrite=True)

Opening raw data file /home/brain/Workspace/meg_driving/meg_preprocess/data/sub-xst/20231013/run1_tsss_mc.fif...
    Range : 18000 ... 341999 =     18.000 ...   341.999 secs
Ready.
Reading 0 ... 323999  =      0.000 ...   323.999 secs...
Overwriting existing file.
Writing /home/brain/Workspace/meg_driving/meg_preprocess/data/sub-xst/20231013/data_run1_tsss_mc.fif
Closing /home/brain/Workspace/meg_driving/meg_preprocess/data/sub-xst/20231013/data_run1_tsss_mc.fif
[done]


#### cut to small data (for testing)

In [ ]:
data_length = 10 # second
start_time = 16 # second

raw_meg_data_file_name = "run1_tsss_mc.fif"
raw_meg_data_file_path_str = os.path.join(data_dir, subject_name, date_string)

raw = mne.io.read_raw_fif(os.path.join(raw_meg_data_file_path_str, raw_meg_data_file_name), preload=True, allow_maxshield=True)
raw = raw.crop(tmin=start_time, tmax=start_time + data_length)

cut_meg_data_file_name = "cut10s_run1_tsss_mc.fif"
raw.save(os.path.join(raw_meg_data_file_path_str, cut_meg_data_file_name), overwrite=True)

### denoise

In [10]:
raw_meg_data_file_name = "cut10s_run1_tsss_mc.fif"
raw_meg_data_file_path_str = os.path.join(data_dir, subject_name, date_string)
raw_filtered = mne.io.read_raw_fif(os.path.join(raw_meg_data_file_path_str, raw_meg_data_file_name), preload=True, allow_maxshield=True)

if not os.path.isdir(os.path.join(preprocessed_subject_path_str,"ICA_denoise")):
    os.mkdir(os.path.join(preprocessed_subject_path_str,"ICA_denoise"))
if not os.path.isdir(os.path.join(preprocessed_subject_path_str,"sensor_prep")):
    os.mkdir(os.path.join(preprocessed_subject_path_str,"sensor_prep"))

# find EOG epochs
try:
    eog_evoked = mne.preprocessing.create_eog_epochs(raw_filtered).average()
    eog_evoked.apply_baseline(baseline=(None, -0.2))
    # eogplot = eog_evoked.plot_joint(show=False)
    # for i in range(len(eogplot)):
    #     eogplot[i].savefig(preprocessed_subject_path_str + "/ICA_denoise/eog" + str(i + 1) + ".tiff")
except:
    print("RuntimeError: No EOG channel(s) found.")

# find ECG epochs
try:
    ecg_evoked = mne.preprocessing.create_ecg_epochs(raw_filtered).average()
    ecg_evoked.apply_baseline(baseline=(None, -0.2))
    # ecgplot = ecg_evoked.plot_joint(show=False)
    # for i in range(len(ecgplot)):
    #     ecgplot[i].savefig(preprocessed_subject_path_str + "/ICA_denoise/ecg" + str(i + 1) + ".tif")
except:
    print("RuntimeError: No ECG channel(s) found.")

# ICA
ica = mne.preprocessing.ICA(n_components=50, method="picard", random_state=0)
ica.fit(raw_filtered)
icaplot = ica.plot_components(title="ICA components", show=False)
for i in range(len(icaplot)):
    icaplot[i].savefig(preprocessed_subject_path_str + "/ICA_denoise/ICAfig" + str(i + 1) + ".tif")

eog_indices, eog_scores = ica.find_bads_eog(raw_filtered)
ecg_indices, ecg_scores = ica.find_bads_ecg(raw_filtered)

ica.exclude = []
ica.exclude = eog_indices + ecg_indices
# ica.exclude = ecg_indices
raw_filtered = ica.apply(raw_filtered)

# save file and records
raw_filtered.save(os.path.join(preprocessed_subject_path_str,"sensor_prep", "preprocessed.fif"), overwrite=True)
# with open(os.path.join(preprocessed_subject_path_str,"03_flag.txt"), "w") as f:
#     f.write("Artefacts cleaned.")


Opening raw data file /home/brain/Workspace/meg_driving/meg_preprocess/data/sub-xst/20231013/cut10s_run1_tsss_mc.fif...
    Range : 34000 ... 44000 =     34.000 ...    44.000 secs
Ready.
Reading 0 ... 10000  =      0.000 ...    10.000 secs...
Using EOG channels: EOG001, EOG002
EOG channel index for this subject is: [0 1]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Selecting channel EOG002 for blink detection
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 10000 samples (10.000 s)

Now detecting blinks and generating corresponding events
Found 2 s

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 10000 samples (10.000 s)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 10000 s

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Using threshold: 0.16 for CTPS ECG detection
Using channel ECG003 to identify heart beats.
Setting up band-pass filter from 8 - 16 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 8.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 7.75 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 16.25 Hz)
- Filter length: 10000 samples (10.000 s)

Number of ECG events detected : 10 (average pulse 59 / min.)
Not setting metadata
10 matching events found
No baseline correction applied
Using data from preloaded Raw for 10 events and 1001 original time points ...
1 bad epochs dropped
Applying ICA to Raw instance
    Transforming to ICA space (50 components)
    Zeroing out 50 ICA components
    Projecting back using 306 PCA components
Overwriting existing file.
Writi

### bem model

In [ ]:
preprocessed_subject_path_str = os.path.join(preprocessed_dir, subject_name)

subject_freesurfer_dir=os.path.join(freesurfer_dir, subject_name)

if not os.path.isdir(preprocessed_subject_path_str + "/fig"):
    os.mkdir(preprocessed_subject_path_str + "/fig")
if not os.path.isdir(preprocessed_subject_path_str + "/src"):
    os.mkdir(preprocessed_subject_path_str + "/src")

Brain = mne.viz.get_brain_class()
brain = Brain(
    subject="",
    views="lat",
    background="w",
    hemi="lh",
    surf="pial",
    subjects_dir=subject_freesurfer_dir,
    title="Surface Reconstruction",
    size=(1000, 800),
)
brain.add_annotation("aparc.a2009s", borders=False)
brain.save_image(preprocessed_subject_path_str + "/fig/recon.png")
brain.close()
# construct the BEM

info = mne.io.read_info(preprocessed_subject_path_str + "/sensor_prep/preprocessed.fif")
conductivity = (0.3,)

model = mne.make_bem_model(
    "", ico=5, conductivity=conductivity, subjects_dir=subject_freesurfer_dir
)
bem_sol = mne.make_bem_solution(model)
bem_model_fig = mne.viz.plot_bem(
    subject="", subjects_dir=subject_freesurfer_dir, orientation="coronal", show=False
)
bem_model_fig.savefig(preprocessed_subject_path_str + "/fig/bem_model_fig.tif")
mne.write_bem_solution(preprocessed_subject_path_str + "/src/bem_sol.h5", bem_sol, overwrite=True)

# automatic coregistreation
fiducials = "estimated"  # get fiducials from fsaverage
coreg = mne.coreg.Coregistration(
    info, subject="", subjects_dir=subject_freesurfer_dir, fiducials=fiducials
)
plot_kwargs = dict(
    subject="",
    subjects_dir=subject_freesurfer_dir,
    surfaces="head-dense",
    dig=True,
    eeg=[],
    meg="sensors",
    show_axes=True,
    coord_frame="meg",
)
coreg.fit_fiducials(verbose=True)
coreg.fit_icp(n_iterations=6, nasion_weight=2.0, verbose=True)
coreg.omit_head_shape_points(distance=5.0 / 1000)
coreg.fit_icp(n_iterations=20, nasion_weight=10.0, verbose=True)
dists = coreg.compute_dig_mri_distances() * 1e3  # in mm
print(
    f"Distance between HSP and MRI (mean/min/max):\n{np.mean(dists):.2f} mm "
    f"/ {np.min(dists):.2f} mm / {np.max(dists):.2f} mm"
)
mne.write_trans(preprocessed_subject_path_str + "/src/auto-trans.fif", coreg.trans, overwrite=True)
vol_src = mne.setup_volume_source_space(
    "",
    subjects_dir=subject_freesurfer_dir,
    surface=subject_freesurfer_dir + "/bem/inner_skull.surf",
    pos=4.0,
    mindist=1.0,
)
mne.write_source_spaces(preprocessed_subject_path_str + "/src/vol-src.fif", vol_src, overwrite=True)
surf_src = mne.setup_source_space(
    "", subjects_dir=subject_freesurfer_dir, spacing="oct6", add_dist="patch"
)
mne.write_source_spaces(preprocessed_subject_path_str + "/src/surf-src.fif", surf_src, overwrite=True)
# plot sources
plot_bem_kwargs = dict(
    subject="",
    subjects_dir=subject_freesurfer_dir,
    brain_surfaces="white",
    orientation="coronal",
    slices=[50, 100, 150, 200],
)
src_fig = mne.viz.plot_bem(src=vol_src, **plot_bem_kwargs, show=False)
src_fig.savefig(preprocessed_subject_path_str + "/fig/vol_src.tif")
src_fig = mne.viz.plot_bem(src=surf_src, **plot_bem_kwargs, show=False)
src_fig.savefig(preprocessed_subject_path_str + "/fig/surf_src.tif")

# save records
# with open(preprocessed_subject_path_str + "/BEM_log.txt", "w") as f:
#     f.write("BEM constructed.")

### source localization

In [ ]:
if not os.path.isdir(preprocessed_subject_path_str + "/fwd_model"):
    os.mkdir(preprocessed_subject_path_str + "/fwd_model")
if not os.path.isdir(preprocessed_subject_path_str + "/src"):
    os.mkdir(preprocessed_subject_path_str + "/src")

# load MEG data and compute the evoked data
vol_src = mne.read_source_spaces(preprocessed_subject_path_str + "/src/vol-src.fif")
surf_src = mne.read_source_spaces(preprocessed_subject_path_str + "/src/surf-src.fif")
trans = mne.read_trans(preprocessed_subject_path_str + "/src/auto-trans.fif")
bem_sol = mne.read_bem_solution(preprocessed_subject_path_str + "/src/bem_sol.h5")
data = mne.io.read_raw_fif(preprocessed_subject_path_str + "/sensor_prep/preprocessed.fif")

# mne.set_eeg_reference(data, projection=True)
cov = mne.compute_raw_covariance(
    data, method=["shrunk", "empirical"], rank="info"
)
fwd_vol = mne.make_forward_solution(
    preprocessed_subject_path_str + "/sensor_prep/preprocessed.fif",
    trans=trans,
    src=vol_src,
    bem=bem_sol,
    meg=True,
    eeg=False,
    n_jobs=20,
)

fwd_surf = mne.make_forward_solution(
    preprocessed_subject_path_str + "/sensor_prep/preprocessed.fif",
    trans=trans,
    src=surf_src,
    bem=bem_sol,
    meg=True,
    eeg=False,
    mindist=3.0,
    n_jobs=20,
)

inv_vol = mne.minimum_norm.make_inverse_operator(
    data.info, fwd_vol, cov, loose="auto", depth=0.8
)
inv_surf = mne.minimum_norm.make_inverse_operator(
    data.info, fwd_surf, cov, loose=0.2, depth=0.8
)

snr = 3.0
method = "MNE"
lambda2 = 1.0 / snr**2

# surface localisation
# volume localisation
# NOTE: volume files were separated into several sub-files to avoid memory error
if not os.path.isdir(preprocessed_subject_path_str + "/src/surf_file"):
    os.mkdir(preprocessed_subject_path_str + "/src/surf_file")
if not os.path.isdir(preprocessed_subject_path_str + "/src/vol_niifile"):
    os.mkdir(preprocessed_subject_path_str + "/src/vol_niifile")
if not os.path.isdir(preprocessed_subject_path_str + "/src/vol_stcfile"):
    os.mkdir(preprocessed_subject_path_str + "/src/vol_stcfile")

In [ ]:
freesurfer_subject_path_dir = os.path.join(freesurfer_dir, subject_name)
freesurfer_fsaverage_path_dir = os.path.join(freesurfer_dir, "fsaverage")

subdata = data.copy()
stc = mne.minimum_norm.apply_inverse_raw(
    subdata,
    inv_surf,
    lambda2,
    method,
)
if os.path.isdir(preprocessed_subject_path_str + "/morph-maps"):
    shutil.rmtree(preprocessed_subject_path_str + "/morph-maps")
morph = mne.compute_source_morph(
    stc,
    subject_from=freesurfer_subject_path_dir,
    subject_to=freesurfer_fsaverage_path_dir,
    src_to=surf_src,
    subjects_dir=preprocessed_subject_path_str,
)
stc_fsaverage = morph.apply(stc)

brain = stc_fsaverage.plot(
    subjects_dir=freesurfer_subject_path_dir,
    initial_time=0.1,
    clim=dict(kind="value", lims=[3, 6, 9]),
    surface="flat",
    hemi="both",
    size=(1000, 500),
    smoothing_steps=5,
    time_viewer=False,
    add_data_kwargs=dict(colorbar_kwargs=dict(label_font_size=10)),
)

# to help orient us, let's add a parcellation (red=auditory, green=motor,
# blue=visual)
brain.add_annotation("HCPMMP1_combined", borders=2)

# You can save a movie like the one on our documentation website with:
# brain.save_movie(time_dilation=20, tmin=0.05, tmax=0.16,
#                  interpolation='linear', framerate=10)

stc_fsaverage.save(
    preprocessed_subject_path_str
    + "/src/surf_file/surf-TS_src",
    ftype="stc",
    overwrite=True,
)  # save a mne-file
del stc  # release memory
del stc_fsaverage

stc = mne.minimum_norm.apply_inverse_raw(
    subdata,
    inv_vol,
    lambda2,
    method,
)  # volume-source TC
stc.save_as_volume(
    preprocessed_subject_path_str
    + "/src/vol_niifile/vol-TS_src"
    + ".nii.gz",
    vol_src,
    dest="mri",
    format="nifti1",
    overwrite=True,
)  # save a nii-file
stc.save(
    preprocessed_subject_path_str
    + "/src/vol_stcfile/vol-TS_src_",
    ftype="stc",
    overwrite=True,
)  # save a mne-file
del subdata
del stc

if os.path.isdir(preprocessed_subject_path_str + "/morph-maps"):
    shutil.rmtree(preprocessed_subject_path_str + "/morph-maps")

### roi_parc 

#### surf

In [ ]:
source_estimate_lh_str = preprocessed_subject_path_str + "/src/surf_file/surf-TS_src-lh.stc"
source_estimate_rh_str = preprocessed_subject_path_str + "/src/surf_file/surf-TS_src-rh.stc"

mne.datasets.fetch_hcp_mmp_parcellation(subjects_dir=freesurfer_dir, accept=True)
if not os.path.isdir(preprocessed_subject_path_str + "/src/ROI_tc"):
    os.mkdir(preprocessed_subject_path_str + "/src/ROI_tc")

surf_src = mne.read_source_spaces(preprocessed_subject_path_str + "/src/surf-src.fif")

# left hemisphere
stc_fsaverage_lh = mne.read_source_estimate(source_estimate_lh_str)
stc_fsaverage_lh.subject = ""
mne.datasets.fetch_hcp_mmp_parcellation(subjects_dir=freesurfer_dir, accept=True)
hcp_mmp_lh = mne.read_labels_from_annot(
    subject="",
    parc="lh.HCPMMP1",
    hemi="lh",
    annot_fname=os.path.join(freesurfer_dir, "fsaverage") + "/label/lh.HCPMMP1.annot",
    subjects_dir=os.path.join(freesurfer_dir, "fsaverage"),
)
ts_lh = mne.extract_label_time_course(
    stcs=stc_fsaverage_lh, labels=hcp_mmp_lh[1:181], src=surf_src
)
scipy.io.savemat(
    preprocessed_subject_path_str
    + "/src/ROI_tc/ts_HCPmmp_lh_"
    + subject_name
    + ".mat",
    {"ts_lh": ts_lh},
)
with open(
    preprocessed_subject_path_str
    + "/src/ROI_tc/lh_ROI_label_"
    + subject_name
    + ".csv",
    "w",
    newline="",
) as csvfile:
    ROIwriter = csv.writer(
        csvfile, delimiter=" ", quotechar="|", quoting=csv.QUOTE_MINIMAL
    )
    for i_ROI in range(1, 181):
        ROIwriter.writerow([hcp_mmp_lh[i_ROI]])

# right hemisphere
stc_fsaverage_rh = mne.read_source_estimate(source_estimate_rh_str)
stc_fsaverage_rh.subject = ""
hcp_mmp_rh = mne.read_labels_from_annot(
    subject="",
    parc="rh.HCPMMP1",
    hemi="rh",
    annot_fname=os.path.join(freesurfer_dir, "fsaverage") + "/label/rh.HCPMMP1.annot",
    subjects_dir=os.path.join(freesurfer_dir, "fsaverage"),
)
ts_rh = mne.extract_label_time_course(
    stcs=stc_fsaverage_rh, labels=hcp_mmp_rh[1:181], src=surf_src
)
scipy.io.savemat(
    preprocessed_subject_path_str
    + "/src/ROI_tc/ts_HCPmmp_rh_"
    + subject_name
    + ".mat",
    {"ts_rh": ts_rh},
)
with open(
    preprocessed_subject_path_str
    + "/src/ROI_tc/rh_ROI_label_"
    + subject_name
    + ".csv",
    "w",
    newline="",
) as csvfile:
    ROIwriter = csv.writer(
        csvfile, delimiter=" ", quotechar="|", quoting=csv.QUOTE_MINIMAL
    )
    for i_ROI in range(1, 181):
        ROIwriter.writerow([hcp_mmp_rh[i_ROI]])

#### vol

In [ ]:
vol_src = mne.read_source_spaces(preprocessed_subject_path_str + "/src/vol-src.fif")
source_estimate_vol_str = preprocessed_subject_path_str + "/src/vol_stcfile/vol-TS_src_-vl.stc"
stc_vol= mne.read_source_estimate(source_estimate_vol_str)

print("check source vol...")
print("vol_src:", vol_src, '\n num:', 39 * 46 * 40)
print("stc_vol:", stc_vol)

In [ ]:
from scipy.io import loadmat

matdic = loadmat(preprocessed_subject_path_str + "/src/ROI_tc/" + "ts_HCPmmp_lh_"+ subject_name+".mat")

ts_lh = np.squeeze(matdic["ts_lh"])
print("ts_lh shape =", ts_lh.shape)

### mask the nii data

#### load atlas brain regions

In [ ]:
import os
# read design matrix
atlas_labels_txt = "Atlas MNI_Glasser_HCP_v1.0, 360 regions.txt"
atlas_labels_file = os.path.join(project_dir, "masks", atlas_labels_txt)

in_file = open(atlas_labels_file,'r')
brain_region_dict={}

# distinguish left and right brain regions
atlas_all_lines = in_file.readlines()

for i_line in range(0,len(atlas_all_lines)):
    if "u:L_" in atlas_all_lines[i_line]:
        
        atlas_brain_region_label = []

        # left brain regions
        atlas_line_left = atlas_all_lines[i_line]
        atlas_temp = atlas_line_left.split(":")
        atlas_brain_region_name = atlas_temp[1][2:]
        # print(atlas_brain_region_name)
        atlas_brain_region_label_left = int(atlas_temp[2][:-1])
        atlas_brain_region_label.append(atlas_brain_region_label_left)
        
        # right brain regions
        atlas_line_right = atlas_all_lines[i_line+1]
        atlas_temp = atlas_line_right.split(":")
        atlas_brain_region_name = atlas_temp[1][2:]
        atlas_brain_region_label_right = int(atlas_temp[2][:-1])
        atlas_brain_region_label.append(atlas_brain_region_label_right)

        # put into dict
        brain_region_dict[atlas_brain_region_name] = atlas_brain_region_label

print(len(brain_region_dict),brain_region_dict)


In [ ]:
brain_region_name_list = list(brain_region_dict.keys())
brain_region_label_list = list(brain_region_dict.values())


#### load hcp mni atlas


In [ ]:
# Load the mask image
mni_glasser_hcp_atlas_name = "MNI_Glasser_HCP_v1.0.nii.gz"
mni_glasser_hcp_atlas_file = os.path.join(project_dir, "masks", mni_glasser_hcp_atlas_name)

mni_glasser_hcp_atlas_mask = nib.load(mni_glasser_hcp_atlas_file)
mni_glasser_hcp_atlas_mask_data = mni_glasser_hcp_atlas_mask.get_fdata()
print("mni_glasser_hcp_atlas_mask_data.shape:", mni_glasser_hcp_atlas_mask_data.shape)

In [ ]:
# load nii data
current_file_str = preprocessed_subject_path_str + "/src/vol_niifile/vol-TS_src.nii.gz"
nii_data = nib.load(current_file_str)
nii_data = nil.image.index_img(nii_data, slice(0, 1))

saved_mask_name = "vol_ts_src.nii.gz"
mask_path = os.path.join(project_dir, "masks", saved_mask_name)
nib.save(nii_data, mask_path)
print("nii_data.shape:", nii_data.shape)

#### resample


In [ ]:
%%bash 
#!/bin/bash
export PATH=$PATH:/home/brain/abin

project_dir="/home/brain/Workspace/meg_driving/preprocess"

3dfractionize -template ${project_dir}/masks/vol_ts_src.nii.gz -input ${project_dir}/masks/MNI_Glasser_HCP_v1.0.nii.gz -clip 0.0 -preserve -prefix ${project_dir}/masks/resampled_mni_mask.nii.gz -overwrite

In [ ]:
# Load the mask image
resampled_mni_mask_name = "resampled_mni_mask.nii.gz"
resampled_mni_mask_file = os.path.join(project_dir, "masks", resampled_mni_mask_name)

resampled_mni_mask = nib.load(resampled_mni_mask_file)
resampled_mni_mask_data = resampled_mni_mask.get_fdata()
print("resampled_mni_mask_data.shape:", resampled_mni_mask_data.shape)

#### create region-based masks


In [ ]:
for i_brain_region_name in np.arange(0, len(brain_region_name_list)):
  
    brain_region_name = brain_region_name_list[i_brain_region_name]

    # mask_name = "resampled_MNI_Glasser_HCP_neural_mask.nii.gz"
    mask_name = "resampled_mni_mask.nii.gz"
    # mask_name = "MNI_Glasser_HCP_v1.0.nii.gz"
    
    mask_file = os.path.join(project_dir, "masks", mask_name)

    # Load the mask image
    mask = nib.load(mask_file)
    mask_data = mask.get_fdata()

    if i_brain_region_name != 0: # remove the effect of first region V1, id V1 is same as mask 0
      mask_data[mask_data==1] = 0

    for i_num_region in brain_region_label_list[i_brain_region_name]:
        mask_data[mask_data==i_num_region] = 1

    mask_data[mask_data!=1] = 0

    voxel_num = np.count_nonzero(mask_data[mask_data!=0])
    print(brain_region_name, voxel_num)

    mask_nifti = nib.Nifti1Image(mask_data, mask.affine, mask.header)

    saved_mask_name = brain_region_name+"_resampled_mni_mask.nii.gz"

    mask_path = os.path.join(project_dir, "masks" , "region_based_masks", saved_mask_name)

    nib.save(mask_nifti, mask_path)


#### mask the nii data

In [ ]:
brain_region_nii_data_list = []

for brain_region_name in brain_region_name_list:

    print('brain_region_name:',brain_region_name)

    ### load mask
    mask_name = brain_region_name+"_resampled_mni_mask.nii.gz"
    mask_file = os.path.join(project_dir, "masks", "region_based_masks", mask_name)

    # Load the mask image
    mask = nib.load(mask_file)

    mask_data = mask.get_fdata()

    voxel_num = np.count_nonzero(mask_data[mask_data!=0])

    print(voxel_num)
    if voxel_num != 0: 

        sub_dir = os.path.join(preprocessed_dir, subject_name)

        # load nii data
        current_file_str = preprocessed_subject_path_str + "/src/vol_niifile/vol-TS_src.nii.gz"
        nii_data = nib.load(current_file_str)

        # mask bold signal
        nifti_masker = nil.input_data.NiftiMasker(mask_img=mask)
        masked_data = nifti_masker.fit_transform(nii_data)
        masked_data = masked_data.T

        # do it for epi data
        masked_data = stats.zscore(masked_data, axis=1, ddof=1)
        masked_data = np.nan_to_num(masked_data)
        # print("masked_data shape (voxel_num,TR_num)", masked_data.shape)

        # add a brain region
        brain_region_nii_data_list.append(masked_data)

    elif voxel_num == 0:

        masked_data = np.empty((0,0), float)

        brain_region_nii_data_list.append(masked_data)

In [ ]:
print("num of brain regions:", len(brain_region_nii_data_list))

for i in range(0, len(brain_region_name_list)):
    print(brain_region_name_list[i],':',brain_region_nii_data_list[i].shape)


#### save and load brain_region_nii_data_list


##### save

In [ ]:
import hdf5storage

matdic = {"brain_region_name_list":brain_region_name_list,
          "brain_region_nii_data_list":brain_region_nii_data_list}
file_dir = os.path.join(preprocessed_dir, subject_name)
hdf5storage.savemat(file_dir+"/brain_region_nii_data_list.mat", matdic)


##### load

In [ ]:
import hdf5storage

file_dir = os.path.join(preprocessed_dir, subject_name)
matdic = hdf5storage.loadmat(file_dir+"/brain_region_nii_data_list.mat", )

brain_region_nii_data_list = matdic["brain_region_nii_data_list"]
brain_region_name_list = matdic["brain_region_name_list"]


In [ ]:
print("num of brain regions:", len(brain_region_nii_data_list))

for i in range(0, len(brain_region_name_list)):
    print(brain_region_name_list[i],':',brain_region_nii_data_list[i].shape)


### check the final results

#### check average of each region

In [ ]:
vol_src = mne.read_source_spaces(preprocessed_subject_path_str + "/src/vol-src.fif")
source_estimate_vol_str = preprocessed_subject_path_str + "/src/vol_stcfile/vol-TS_src_-vl.stc"
stc_vol= mne.read_source_estimate(source_estimate_vol_str)

print("check source vol...")
print("vol_src:", vol_src, '\n num:', 39 * 46 * 40)
print("stc_vol:", stc_vol)

In [ ]:
from scipy.io import loadmat

matdic = loadmat(preprocessed_subject_path_str + "/src/ROI_tc/" + "ts_HCPmmp_lh_sub-test.mat")

ts_lh = np.squeeze(matdic["ts_lh"])
print("ts_lh shape =", ts_lh.shape)

#### check each region with mask

##### load

In [ ]:
import hdf5storage

file_dir = os.path.join(preprocessed_dir, subject_name)
matdic = hdf5storage.loadmat(file_dir+"/brain_region_nii_data_list.mat", )

brain_region_nii_data_list = matdic["brain_region_nii_data_list"]
brain_region_name_list = matdic["brain_region_name_list"]


In [ ]:
print("num of brain regions:", len(brain_region_nii_data_list))

for i in range(0, len(brain_region_name_list)):
    print(brain_region_name_list[i],':',brain_region_nii_data_list[i].shape)
